In [1]:
import numpy as np
import tensorflow as tf
from random import randint
import scipy as scipy
import scipy.interpolate as interpolate
import math
from random import *
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from scipy.stats import logistic
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt2

import tflib as lib
import tflib.ops.linear
import tflib.ops.conv2d
import tflib.ops.batchnorm
import tflib.ops.deconv2d
import tflib.save_images
import tflib.plot


Lambda = 10.0
BATCH_SIZE = 56
BATCH_SIZE_2 = 100
dimensionality = 30
num_epochs = 80
num_iters = 1000
k=2
max_grad_norm = 1000
gamma = 1.0

# main parameters

C=10.0

# additional parameters
sigma=0.8

number_of_neurons_regr = 50
number_of_points = 569-65
number_of_neurons = number_of_points

/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
breast = load_breast_cancer(return_X_y=False)
data = []
target = []
rr = np.random.permutation(569)
for x in rr:
    data.append(breast['data'][x])
    target.append(breast['target'][x])
# training data
_train_x = np.array(data)
_train_y = np.array(target)
print(_train_x.shape)
print(_train_y.shape)
rho = 0.5

(569, 30)
(569,)


In [3]:
new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
Dataplace = tf.placeholder(tf.float32, shape=(dimensionality, number_of_points))
Data = tf.placeholder(tf.float32, shape=(BATCH_SIZE, dimensionality))
outputs = tf.placeholder(tf.float32, shape=(BATCH_SIZE,1)) 

tf_data_x = gamma*tf.random_normal([BATCH_SIZE, dimensionality]) # аргументы функции
tf_data_y = tf.reduce_mean(tf.math.cos(tf.matmul(tf_data_x, Dataplace)), axis=1) # значения функции

tf_data_w = tf.placeholder(tf.float32, shape=(number_of_neurons,1))
tf_data_B = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons))
tf_data_P = tf.placeholder(tf.float32, shape=(dimensionality, dimensionality))

tf_w_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr,1))
tf_B_regr = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons_regr))
tf_bias_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr))


tf_data_second = gamma*tf.random_normal([BATCH_SIZE_2, dimensionality])
Lbd = tf.placeholder(tf.float32, shape=[], name="lambda")

# characteristic function parameters
w = tf.Variable(tf.random_normal([number_of_neurons,1], stddev=0.0), name="neuron_weights")
B = tf.Variable(initial_value=Dataplace, name="weights")
# regression function parameters
w_regr = tf.Variable(tf.random_normal([number_of_neurons_regr,1], stddev=0.35), name="neuron_weights")
B_regr = tf.Variable(tf.random_normal([dimensionality,number_of_neurons_regr], stddev=0.35), name="weights")
bias_regr = tf.Variable(tf.random_normal([number_of_neurons_regr], stddev=0.0), name="biases")

prediction = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_x, B)), tf.nn.sigmoid(w))
penalty = tf.square((2/number_of_neurons)*tf.reduce_sum(tf.nn.sigmoid(w))-1.0)

out_loss = (1-rho)*tf.reduce_mean(tf.square(prediction - tf_data_y)) + C*penalty

prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, B)), tf.nn.sigmoid(w))
prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, B_regr), bias_regr)), w_regr)

grad_psi = tf.reshape(tf.gradients(prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
grad_psi_regr = tf.reshape(tf.gradients(prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

tf_prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, tf_data_B)), tf.nn.sigmoid(tf_data_w))
tf_prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, tf_B_regr), tf_bias_regr)), tf_w_regr)

tf_data_grad_psi = tf.reshape(tf.gradients(tf_prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
tf_data_grad_psi_regr = tf.reshape(tf.gradients(tf_prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

old_part = tf.matmul(tf_data_grad_psi, tf_data_P)
old_part_regr = tf.matmul(tf_data_grad_psi_regr, tf_data_P)

loss = out_loss + Lbd*(1-rho)*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi, old_part)), axis=1)) + Lbd*rho*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi_regr, old_part_regr)), axis=1))

#offset = tf.random.uniform(shape=[], minval=0, maxval=12, dtype=tf.int32)*BATCH_SIZE
x_plus_error = Data+sigma*tf.random_normal([BATCH_SIZE, dimensionality])
regression = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(x_plus_error, B_regr), bias_regr)), w_regr)
sdr = -tf.reduce_mean(tf.multiply(regression, outputs)) + tf.reduce_mean(tf.math.log(1.0+tf.math.exp(regression)))

loss_hybrid = loss + rho*sdr

In [4]:
target = tf.train.AdamOptimizer(learning_rate=3e-5, beta1=0.5, beta2=0.9).minimize(loss_hybrid)

In [5]:
def sigmoid(x):
  return 1. /(1+np.exp(-x))

In [6]:
def euclidean_distance(pnt1, pnt2):
    '''Finds the distance between 2 points: pnt1, pnt2'''
    # element-wise computations are automatically handled by numpy
    return sum((pnt1 - pnt2) ** 2)

from collections import defaultdict

def find_majority(labels):
    '''Finds the majority class/label out of the given labels'''
    # defaultdict(type) is to automatically add new keys without throwing error.
    counter = defaultdict(int)
    for label in labels:
        counter[label] += 1

    # Finding the majority class.
    majority_count = max(counter.values())
    for key, value in counter.items():
        if value == majority_count:
            return key

def new_predict(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return find_majority(k_labels)

def new_predict_regr(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return np.mean(k_labels)

In [ ]:
scores = []
scores3 = []
for part in range(8):
    print("Current part is %d\n" % part)
    train_x = np.concatenate((_train_x[0:(part*65)], _train_x[(part+1)*65:569]), axis=0)
    train_y = np.concatenate((_train_y[0:(part*65)], _train_y[(part+1)*65:569]), axis=0)
    test_x = _train_x[part*65:(part+1)*65]
    test_y = _train_y[part*65:(part+1)*65]
    scaler = preprocessing.StandardScaler().fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)
    transpose_train_x = np.transpose(train_x)

    cur_w = np.random.normal(0, 0.35, (number_of_neurons,1))
    cur_B =  np.random.normal(0, 0.35, (dimensionality, number_of_neurons))
    cur_biases = np.zeros((number_of_neurons))
    cur_P = np.zeros((dimensionality, dimensionality)) 
    cur_w_regr = np.random.normal(0, 0.35, (number_of_neurons_regr,1))
    cur_B_regr = np.random.normal(0, 0.35, (dimensionality, number_of_neurons_regr))
    cur_bias_regr = np.random.normal(0, 0.35, (number_of_neurons_regr))
    
    O = np.zeros((dimensionality, k)) 
    cur_iter = 0
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init, feed_dict={Dataplace: transpose_train_x})
    
    for epoch in range(num_epochs):
        print("Epoch %d" %(epoch))
        for iteration in range(num_iters):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            sess.run(target, feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x,
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
        reses = []
        for i in range(100):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            res = sess.run([loss_hybrid], feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x, 
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
            reses.append(res)
        print ("Iter %d: loss: %.4f\n" %(cur_iter, np.mean(np.array(reses))))
        lib.plot.plot('train cost', np.mean(np.array(reses)))

        cur_w, cur_B, cur_w_regr, cur_B_regr, cur_bias_regr = sess.run([w, B, w_regr, B_regr, bias_regr])

        third_grad_psi = np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))
        third_grad_psi_regr = np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))
        for r in range(1000):
            sess.run([tf_data_second])
            np.concatenate((third_grad_psi, np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))), axis=0) 
            np.concatenate((third_grad_psi_regr, np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))), axis=0)

        M = (1-rho)*np.matmul(np.transpose(third_grad_psi), third_grad_psi)+\
            rho*np.matmul(np.transpose(third_grad_psi_regr), third_grad_psi_regr)
        u, s, vh = np.linalg.svd(M, full_matrices=True)
        O = u[:,0:k:1]
        print(s)
        cur_P = np.matmul(O, np.transpose(O))
        tvr = 1-np.sum(np.multiply(s[0:k],s[0:k]))/np.sum(np.multiply(s,s))
        lib.plot.plot('total variance to retain', tvr)
        lib.plot.tick()
        lib.plot.flush()
    reduced_train_x = np.matmul(train_x, O)
    reduced_test_x = np.matmul(test_x, O)
    clf = LogisticRegression(random_state=0).fit(reduced_train_x, train_y)
    score = clf.score(reduced_test_x, test_y)
    scores.append(score)
    print ("Part %d: rate on test %.4f\n" %(part, score))
    test_rgb = []
    for h in test_y:
        if h == 1:
            test_rgb.append('r')
        else:
            test_rgb.append('b')
    plt2.close()
    plt2.scatter(np.transpose(reduced_test_x)[0], np.transpose(reduced_test_x)[1], alpha=0.2, c=np.array(test_rgb))
    plt2.xlabel("1st component")
    plt2.ylabel("2nd component")
    plt2.show()
    sess.close()
    print(cur_P)

    i = 0
    total_correct = 0
    for test_image in reduced_test_x:
        pred = new_predict(10, reduced_train_x, train_y, test_image)
        if pred == test_y[i]:
            total_correct += 1
        score3 = (total_correct / (i+1)) * 100            
        i += 1
    print('acc:', str(round(score3, 2))+'%')
    scores3.append(score3)

print ("Average rate on test %.4f\n" %(np.mean(np.array(scores))))
print ("Average 10-NN acc on test %.4f\n" %(np.mean(np.array(scores3))))

Current part is 0

Epoch 0
Iter 1100: loss: 10.6162

[26.451145   15.300708   11.768134    9.265918    8.797858    6.078735
  4.240258    3.7700136   3.2244456   2.2158964   1.9548602   1.590778
  1.4978124   1.3853097   1.1312615   1.0505191   0.9506182   0.6341434
  0.56424135  0.49286655  0.40658563  0.3432738   0.29467338  0.24772355
  0.20750587  0.15562351  0.13662776  0.11615233  0.09892901  0.0702045 ]
iter 1	train cost	10.616240501403809	total variance to retain	0.30069243907928467
Epoch 1
Iter 2200: loss: 5.4034

[21.7769     11.627873   10.264717    7.75609     6.429472    4.6648884
  3.2958965   2.3296583   2.1756048   1.909525    1.737791    1.4957298
  1.2459792   0.8342622   0.77850634  0.74018985  0.6589072   0.5626897
  0.38817304  0.34455165  0.2715573   0.24300097  0.20095457  0.16860205
  0.15111725  0.11247508  0.09530371  0.07984574  0.07348751  0.04688804]
iter 2	train cost	5.4033684730529785	total variance to retain	0.3016374111175537
Epoch 2
Iter 3300: loss: 4.

Iter 18700: loss: 0.3493

[1.54088526e+01 9.26286876e-01 3.57707679e-01 2.43378624e-01
 1.26016304e-01 7.32099488e-02 6.54434264e-02 5.30575700e-02
 3.72064412e-02 3.11474912e-02 2.43688859e-02 2.10799910e-02
 1.73746571e-02 1.52987605e-02 1.28815835e-02 1.23762833e-02
 1.13025103e-02 8.96058138e-03 8.16213712e-03 6.61623850e-03
 4.22155671e-03 3.87768983e-03 3.27823800e-03 2.88829487e-03
 2.62670498e-03 1.92819419e-03 1.30222563e-03 1.13856373e-03
 1.06091576e-03 7.93162792e-04]
iter 17	train cost	0.34925830364227295	total variance to retain	0.0009230375289916992
Epoch 17
Iter 19800: loss: 0.3391

[1.79994164e+01 1.22393548e+00 4.54639524e-01 2.87541091e-01
 1.75691992e-01 1.09122895e-01 7.45758042e-02 5.31475917e-02
 3.56105417e-02 2.94849817e-02 2.41470635e-02 2.05117911e-02
 1.62803680e-02 1.51199251e-02 1.40262553e-02 9.37859807e-03
 8.41613486e-03 7.92658981e-03 7.32875708e-03 6.03190297e-03
 3.77302687e-03 3.58464639e-03 3.43811722e-03 2.70477240e-03
 2.44325260e-03 2.01835320e-

Iter 35200: loss: 0.2249

[2.6521061e+01 1.2424796e+00 1.9201313e-01 1.6488983e-01 1.3156673e-01
 9.9559128e-02 7.4872062e-02 4.5226432e-02 2.6797628e-02 2.4254216e-02
 1.8694839e-02 1.6865002e-02 1.3147608e-02 1.2293542e-02 8.8964039e-03
 6.9157816e-03 4.3528681e-03 3.7466653e-03 3.4395801e-03 3.1135431e-03
 2.5028703e-03 2.1540404e-03 1.8638951e-03 1.5328020e-03 1.4888860e-03
 1.2524114e-03 8.0455269e-04 7.2916556e-04 4.6284113e-04 3.7694621e-04]
iter 32	train cost	0.2248784899711609	total variance to retain	0.00014382600784301758
Epoch 32
Iter 36300: loss: 0.2228

[2.6597078e+01 1.3174642e+00 2.0537826e-01 1.4885175e-01 1.2351540e-01
 8.4939025e-02 7.4946374e-02 5.1529415e-02 3.0753229e-02 2.6237890e-02
 2.0888157e-02 1.7706970e-02 1.6049147e-02 1.3310082e-02 9.7434912e-03
 8.5715940e-03 5.2382178e-03 3.9290967e-03 3.0562342e-03 2.7927719e-03
 2.2176174e-03 2.2008610e-03 1.7071842e-03 1.4331414e-03 1.2530413e-03
 1.1126443e-03 9.1543817e-04 7.0194696e-04 4.8163446e-04 2.6603413e-04]

Iter 51700: loss: 0.1799

[3.6380028e+01 9.4725865e-01 1.4174657e-01 1.0870541e-01 8.2525142e-02
 6.0762536e-02 4.5256417e-02 3.5160895e-02 2.2863442e-02 2.0098127e-02
 1.5885154e-02 1.4542464e-02 1.1817957e-02 9.4860960e-03 7.8375293e-03
 6.5548904e-03 3.3599490e-03 3.0364338e-03 2.9037965e-03 2.1542918e-03
 1.7945357e-03 1.4972438e-03 1.4306555e-03 1.0671329e-03 9.0076640e-04
 7.4122538e-04 6.2374433e-04 4.4852341e-04 3.5666034e-04 1.9199528e-04]
iter 47	train cost	0.1798686385154724	total variance to retain	3.5762786865234375e-05
Epoch 47
Iter 52800: loss: 0.1803

[3.5525795e+01 1.0285814e+00 1.5897010e-01 1.0940716e-01 8.3474033e-02
 7.3492743e-02 4.0229555e-02 3.5518933e-02 3.2647956e-02 2.3075389e-02
 1.6493235e-02 1.3856041e-02 1.2210917e-02 1.1022483e-02 8.9972494e-03
 7.4972841e-03 3.9356607e-03 3.1356139e-03 2.6801995e-03 2.1090377e-03
 1.7397894e-03 1.6689140e-03 1.4328785e-03 1.2231314e-03 1.1178391e-03
 8.7635859e-04 6.9557037e-04 4.8436888e-04 3.2823850e-04 1.8549587e-04]

Iter 68200: loss: 0.1570

[4.30333710e+01 1.01517022e+00 1.50086030e-01 1.02360554e-01
 7.42001161e-02 5.13925180e-02 3.44857983e-02 2.74380296e-02
 2.50838585e-02 2.03341451e-02 1.73846539e-02 1.49217974e-02
 1.41415512e-02 1.00853397e-02 8.49597529e-03 5.60782198e-03
 3.61949229e-03 3.42564750e-03 2.93249195e-03 2.04869010e-03
 1.59181282e-03 1.53568340e-03 1.20328611e-03 1.00213988e-03
 9.52226867e-04 7.77474896e-04 5.58893371e-04 3.87855922e-04
 3.05666472e-04 1.70933796e-04]
iter 62	train cost	0.1570308357477188	total variance to retain	2.4378299713134766e-05
Epoch 62
Iter 69300: loss: 0.1556

[4.2998375e+01 9.6671855e-01 1.3238408e-01 9.4731160e-02 7.0484780e-02
 5.1368084e-02 3.9713912e-02 2.9944265e-02 2.7087940e-02 1.5193489e-02
 1.3375401e-02 1.1537392e-02 1.0450213e-02 9.5267510e-03 8.5495310e-03
 6.2015946e-03 3.7980885e-03 3.2665508e-03 2.3096793e-03 1.7325676e-03
 1.6479507e-03 1.4268877e-03 1.3048325e-03 1.1735631e-03 9.2266256e-04
 7.2597939e-04 6.4243202e-04 3.5881271e